<a href="https://colab.research.google.com/github/dljaimes/AdvancedMethodsinDataAnalysis1Grupo_5/blob/main/E_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
!pip install -q wordcloud
!pip install livelossplot==0.1.2
import pandas as pd
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
#from livelossplot import PlotLossesKeras
import keras
from keras import backend as K
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
from livelossplot import PlotLossesKeras

In [38]:
# Se descargan los Stopwords:

nltk.corpus.stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
# Cargue del dataset:

dataTraining = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)

In [41]:
# Se quitan caractéres especiales sobre cada texto:

def pre_process(text):
    # lowercase
    text = text.lower()
    # tags
    text = re.sub('&lt;/?.*?&gt;',' &lt;&gt; ',text)
    # special characters and digits
    text=re.sub('(\\d|\\W)+',' ',text)
    # remove punctuation
    #text = re.sub('[.;:!\'?,\"()\[\]]', '', text)
    #text = [REPLACE.sub('', line) for line in text]
    
    return text

# Se frea una columna con 
dataTraining['plot_low']=dataTraining['plot'].apply(lambda x:pre_process(x))




In [45]:
# Se quitan los stopwords:

english_stop_words=stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

dataTraining['plot_low_rm'] = remove_stop_words(dataTraining['plot_low'])


from nltk.stem.porter import PorterStemmer
def get_stemmed_text(corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

dataTraining['plot_low_rm_stem'] = get_stemmed_text(dataTraining['plot_low_rm'])

In [46]:
dataTraining.columns

Index(['year', 'title', 'plot', 'genres', 'rating', 'plot_low', 'plot_low_rm',
       'plot_low_rm_stem'],
      dtype='object')

In [47]:
dataTraining[['plot','plot_low','plot_low_rm_stem']]

,plot,plot_low,plot_low_rm_stem
3107,most is the story of a single father who takes...,most is the story of a single father who takes...,stori singl father take eight year old son wor...
900,a serial killer decides to teach the secrets o...,a serial killer decides to teach the secrets o...,serial killer decid teach secret satisfi caree...
6724,"in sweden , a female blackmailer with a disfi...",in sweden a female blackmailer with a disfigur...,sweden femal blackmail disfigur facial scar me...
4704,"in a friday afternoon in new york , the presi...",in a friday afternoon in new york the presiden...,friday afternoon new york presid tredway corpo...
2582,"in los angeles , the editor of a publishing h...",in los angeles the editor of a publishing hous...,lo angel editor publish hous carol hunnicut go...
...,...,...,...
8417,""" our marriage , their wedding . "" it ' s l...",our marriage their wedding it s lesson number...,marriag wed lesson number one newli engag coup...
1592,"the wandering barbarian , conan , alongside ...",the wandering barbarian conan alongside his go...,wander barbarian conan alongsid goofi rogu pal...
1723,"like a tale spun by scheherazade , kismet fol...",like a tale spun by scheherazade kismet follow...,like tale spun scheherazad kismet follow remar...
7605,"mrs . brisby , a widowed mouse , lives in a...",mrs brisby a widowed mouse lives in a cinder b...,mr brisbi widow mous live cinder block childre...


In [49]:
# Se separan las variables de salida (y) y el texto transformado (sin stopwords y stemmizado):

plots = dataTraining['plot_low_rm_stem']
y = (dataTraining['rating'] >= dataTraining['rating'].mean()).astype(int) # se discretiza la variable 1 si la calificacion es >= promedio 0 dlc.

In [50]:
y

3107    1
900     0
6724    1
4704    1
2582    1
       ..
8417    0
1592    0
1723    0
7605    1
215     1
Name: rating, Length: 7895, dtype: int64

In [72]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [69]:
# Tokenización por palabra:

count_ = Tokenizer()
count_.fit_on_texts(plots)

In [83]:
vocabulary = count_.word_index

In [84]:
'major' in vocabulary.keys()

True

In [77]:
# Se transforma cada texto en palabras de acuerdo con su diccionario de ínidices:

frases_num = count_.sequences_to_texts(plots)
matriz_pad =  pad_sequences(frases_num)

In [78]:
matriz_pad

array([], shape=(7895, 0), dtype=int32)

In [ ]:
aaa

In [51]:
# Se crea diccionario con el índice de cada letra/palabra:

voc = set(''.join(plots))
vocabulary = {x: idx + 1 for idx, x in enumerate(set(voc))}

In [ ]:
vocabulary

In [82]:
plots[0].split()[:10]

['major',
 'benson',
 'winifr',
 'payn',
 'discharg',
 'marin',
 'payn',
 'killin',
 'machin',
 'war']

In [100]:
max_url_len = 540
X = [x[:max_url_len] for x in plots]
# Convert characters to int and pad
X = [[vocabulary[x1] for x1 in x if x1 in vocabulary.keys()] for x in plots.str.split()]

In [91]:
X[0][:10]

[39, 629, 22, 12, 1046, 8, 31, 45, 17, 2744]

In [92]:
dataTraining[['plot','plot_low','plot_low_rm_stem']].head(10)

,plot,plot_low,plot_low_rm_stem
3107,most is the story of a single father who takes...,most is the story of a single father who takes...,stori singl father take eight year old son wor...
900,a serial killer decides to teach the secrets o...,a serial killer decides to teach the secrets o...,serial killer decid teach secret satisfi caree...
6724,"in sweden , a female blackmailer with a disfi...",in sweden a female blackmailer with a disfigur...,sweden femal blackmail disfigur facial scar me...
4704,"in a friday afternoon in new york , the presi...",in a friday afternoon in new york the presiden...,friday afternoon new york presid tredway corpo...
2582,"in los angeles , the editor of a publishing h...",in los angeles the editor of a publishing hous...,lo angel editor publish hous carol hunnicut go...
9784,two drifters are passing through a western tow...,two drifters are passing through a western tow...,two drifter pass western town news come local ...
8804,two gay men on a date are murdered the night b...,two gay men on a date are murdered the night b...,two gay men date murder night halloween west h...
3501,adult huw morgan is about to move away from th...,adult huw morgan is about to move away from th...,adult huw morgan move away south wale coal min...
7145,bertram pincus is a man whose people skills le...,bertram pincus is a man whose people skills le...,bertram pincu man whose peopl skill leav much ...
4067,roy munson was raised to be the best bowler in...,roy munson was raised to be the best bowler in...,roy munson rais best bowler world train earli ...


In [ ]:
vocabulary[]

In [99]:
dataTraining['plot_low_rm_stem'].str.len().describe()

count    7895.000000
mean      424.891577
std       275.441225
min         0.000000
25%       236.000000
50%       371.000000
75%       540.000000
max      4930.000000
Name: plot_low_rm_stem, dtype: float64

In [18]:
len(dataTraining['plot'].tolist())

7895

In [101]:
X_pad = sequence.pad_sequences(X, maxlen=max_url_len)
X_pad

array([[    0,     0,     0, ...,  3760,  6588,    69],
       [    0,     0,     0, ...,  1100,   490,  1646],
       [    0,     0,     0, ...,     8,    31,  1773],
       ...,
       [    0,     0,     0, ...,   616,    12,   196],
       [    0,     0,     0, ...,    24, 17083,   105],
       [    0,     0,     0, ..., 13014,  3529,  1484]], dtype=int32)

In [102]:
X_pad.shape

(7895, 540)

In [143]:

model = Sequential()
model.add(Embedding(len(vocabulary) + 1, 128, input_length=max_url_len))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 540, 128)          3460864   
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 3,481,505
Trainable params: 3,481,505
Non-trainable params: 0
_________________________________________________________________


In [118]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.33, random_state=42)

In [121]:
model.fit(X_train, y_train, validation_data=[X_test, y_test], 
          batch_size=128, epochs=10, verbose=1)
          #callbacks=[PlotLossesKeras()])

Epoch 1/10
42/42 [==============================] - 67s 976ms/step - loss: 0.6911 - accuracy: 0.5305 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
42/42 [==============================] - 40s 946ms/step - loss: 0.6166 - accuracy: 0.6940 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
42/42 [==============================] - 40s 952ms/step - loss: 0.4551 - accuracy: 0.8308 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
42/42 [==============================] - 40s 952ms/step - loss: 0.3170 - accuracy: 0.8890 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
42/42 [==============================] - 40s 949ms/step - loss: 0.1768 - accuracy: 0.9475 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
42/42 [==============================] - 40s 952ms/step - loss: 0.1171 - accuracy: 0.9727 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/10
42/42 [==============================] - 40s 962ms/step - loss: 0.0648 - accuracy: 

In [122]:
X_train.shape

(5289, 540)

In [123]:
X_test.shape

(2606, 540)

In [125]:
from sklearn.metrics import accuracy_score

In [139]:
pred_ = model.predict_classes(X_pad)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [140]:
y_real = np.array(y).reshape(-1,1)

# Accuracy sobre toda la base

In [141]:
accuracy_score(pred_, y_real)

0.8635845471817606

In [142]:
# Se calcula el accuracy sobre la base de Test:

pred_test = model.predict_classes(X_test)

accuracy_score(pred_test, y_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.5897927858787414